In [1]:
import os
import sys
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# 获取当前工作目录
current_path = os.getcwd()

# 获取当前目录的最后一个字符串（即最后一个文件夹名）
last_directory = os.path.basename(current_path)

# 检查是否是 notebook
if last_directory == 'notebooks':
    # 切换到上一级目录
    os.chdir('..')

#print(f"切换到目录: {os.getcwd()}")

sys.path.append('src')  # 将模块所在的路径添加到 sys.path

from util.logconf import logging
from app.infer.eval_seg import SegmentationTestingApp

log = logging.getLogger('nb')

2025-05-21 08:36:23.707343: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-21 08:36:23.707398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-21 08:36:23.708738: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-21 08:36:23.717046: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-21 08:36:24.522363: W tensorflow/compiler/tf2

In [2]:
# 设置随机数种子方便复现实验结果
def set_random_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #torch.set_num_threads(1)

set_random_seed(42)

In [3]:
# 定义一个辅助函数用于启动应用
def run(app, *argv):
    argv = list(argv)
    log.info("Running: {}({!r}).main()".format(app.__name__, argv))
    app(argv).main()
    log.info("Finished: {}.{!r}).main()".format(app.__name__, argv))

In [4]:
run(SegmentationTestingApp, "--num-workers=1", 
    "--platform=pytorch",
    "--model-path=data/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state"
)

2025-05-20 23:50:53,084 INFO     pid:3399200 nb:004:run Running: SegmentationTestingApp(['--num-workers=1', '--platform=pytorch', '--model-path=data/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state']).main()
2025-05-20 23:50:53,420 INFO     pid:3399200 app.infer.testing_seg:143:initPytorchModel Using CUDA; 1 devices.
2025-05-20 23:50:53,434 INFO     pid:3399200 app.infer.testing_seg:172:main Starting SegmentationTestingApp, Namespace(platform='pytorch', model_path='data/models/seg/seg_2025-04-30_18.55.29_seg.3500000.state', verbose=False, target=None, device_id=None, batch_size=16, num_workers=1)
2025-05-20 23:50:59,668 INFO     pid:3399200 core.dsets_seg:273:__init__ <core.dsets_seg.Luna2dSegmentationDataset object at 0x7ff8e022d640>: 89 validation series, 1122 slices, 154 nodules
2025-05-20 23:50:59,669 INFO     pid:3399200 app.infer.testing_seg:178:main 71 batches of size 16*1
2025-05-20 23:50:59,682 WARNING  pid:3399200 util.util:221:enumerateWithEstimate E0 Validation  ----/71

In [5]:
run(SegmentationTestingApp, "--num-workers=1", 
    "--platform=rknn",
    "--model-path=deployment/models/seg_model.rknn",
    "--target=rk3588",
    "--verbose"
)

2025-05-20 23:56:11,903 INFO     pid:3399200 nb:004:run Running: SegmentationTestingApp(['--num-workers=1', '--platform=rknn', '--model-path=deployment/models/seg_model.rknn', '--target=rk3588', '--verbose']).main()
I rknn-toolkit2 version: 2.3.2
adb: unable to connect for root: closed
I target set by user is: rk3588


--> Init runtime environment


I Get hardware info: target_platform = rk3588, os = Linux, aarch = aarch64
I Check RK3588 board npu runtime version
I Starting ntp or adb, target is RK3588
I Start adb...
I Connect to Device success!


I NPUTransfer(3399200): Starting NPU Transfer Client, Transfer version 2.2.2 (12abf2a@2024-09-02T03:22:41)
I NPUTransfer(3399200): TransferBuffer: min aligned size: 1024
D RKNNAPI: ==============================================
D RKNNAPI: RKNN VERSION:
D RKNNAPI:   API: 2.3.2 (1842325 build@2025-03-30T09:55:23)
D RKNNAPI:   DRV: rknn_server: 2.3.2 (1842325 build@2025-03-30T09:54:34)
D RKNNAPI:   DRV: rknnrt: 2.3.2 (429f97ae6b@2025-04-09T09:09:27)
D RKNNAPI: ==============================================


2025-05-20 23:56:14,446 INFO     pid:3399200 app.infer.testing_seg:172:main Starting SegmentationTestingApp, Namespace(platform='rknn', model_path='deployment/models/seg_model.rknn', verbose=True, target='rk3588', device_id=None, batch_size=1, num_workers=1)
2025-05-20 23:56:14,612 INFO     pid:3399200 core.dsets_seg:273:__init__ <core.dsets_seg.Luna2dSegmentationDataset object at 0x7ff8e022da00>: 89 validation series, 1122 slices, 154 nodules
2025-05-20 23:56:14,613 INFO     pid:3399200 app.infer.testing_seg:178:main 1122 batches of size 1*1
2025-05-20 23:56:14,615 WARNING  pid:3399200 util.util:221:enumerateWithEstimate E0 Validation  ----/1122, starting


D RKNNAPI: Input tensors:
D RKNNAPI:   index=0, name=input, n_dims=4, dims=[1, 512, 512, 7], n_elems=1835008, size=3670016, w_stride = 0, size_with_stride = 0, fmt=NHWC, type=FP16, qnt_type=NONE, zp=0, scale=1.000000
D RKNNAPI: Output tensors:
done
Model-deployment/models/seg_model.rknn is rknn model, starting val
D RKNNAPI:   index=0, name=output, n_dims=4, dims=[1, 1, 512, 512], n_elems=262144, size=524288, w_stride = 0, size_with_stride = 0, fmt=NCHW, type=FP16, qnt_type=NONE, zp=0, scale=1.000000


2025-05-20 23:56:34,090 INFO     pid:3399200 util.util:238:enumerateWithEstimate E0 Validation     4/1122, done at 2025-05-21 00:25:48, 0:29:20
2025-05-20 23:56:51,553 INFO     pid:3399200 util.util:238:enumerateWithEstimate E0 Validation    16/1122, done at 2025-05-21 00:24:11, 0:27:43
2025-05-20 23:58:40,680 INFO     pid:3399200 util.util:238:enumerateWithEstimate E0 Validation    64/1122, done at 2025-05-21 00:35:15, 0:38:47
2025-05-21 00:02:48,933 INFO     pid:3399200 util.util:238:enumerateWithEstimate E0 Validation   256/1122, done at 2025-05-21 00:24:16, 0:27:48
2025-05-21 00:22:28,910 INFO     pid:3399200 util.util:238:enumerateWithEstimate E0 Validation  1024/1122, done at 2025-05-21 00:24:56, 0:28:28
2025-05-21 00:24:54,128 WARNING  pid:3399200 util.util:251:enumerateWithEstimate E0 Validation  ----/1122, done at 2025-05-21 00:24:54
2025-05-21 00:24:54,130 INFO     pid:3399200 app.infer.testing_seg:258:logMetrics E0 SegmentationTestingApp
2025-05-21 00:24:54,131 INFO     pid: